In [ ]:
# Notebook which iterates through a folder, including subfolders, 
# and convert DICOM files to AVI files of a defined size (natively 112 x 112)

import re
import os, os.path
from os.path import splitext
import pydicom as dicom
import numpy as np
from pydicom.uid import UID, generate_uid
import shutil
from multiprocessing import dummy as multiprocessing
import time
import subprocess
import datetime
from datetime import date
import sys
import cv2
#from scipy.misc import imread
import matplotlib.pyplot as plt
import sys
from shutil import copy
import math

In [ ]:
AllA4cNames = "dicom_data/"
destinationFolder = "des/"  
cropSize = (112, 112)

In [ ]:
# Dependencies you might need to run code
# Commonly missing

#!pip install pydicom
#!pip install opencv-python

In [ ]:
def mask(output):
    dimension = output.shape[0]
    
    # Mask pixels outside of scanning sector
    m1, m2 = np.meshgrid(np.arange(dimension), np.arange(dimension))
    

    mask = ((m1+m2)>int(dimension/2) + int(dimension/10)) 
    mask *=  ((m1-m2)<int(dimension/2) + int(dimension/10))
    mask = np.reshape(mask, (dimension, dimension)).astype(np.int8)
    maskedImage = cv2.bitwise_and(output, output, mask = mask)
    
    #print(maskedImage.shape)
    
    return maskedImage



In [ ]:
def makeVideo(fileToProcess, destinationFolder, AllA4cNames):
    try:
        # 移除副檔名，得到純檔名
        fileName = os.path.splitext(fileToProcess.split('\\')[-1])[0]
        
        # 計算相對路徑並在 destinationFolder 中建立相同的目錄結構
        relative_path = os.path.relpath(fileToProcess, AllA4cNames)
        output_path = os.path.join(destinationFolder, os.path.dirname(relative_path))
        os.makedirs(output_path, exist_ok=True)

        # 設定影片的完整檔案路徑
        video_filename = os.path.join(output_path, fileName + '.avi')
        if os.path.exists(video_filename):
            print(f"{video_filename} has already been processed")
            return
        
        # Load DICOM file
        try:
            dataset = dicom.dcmread(fileToProcess, force=True)
            testarray = dataset.pixel_array
            
            # 顯示 testarray 的形狀
            print(f"Shape of pixel array for {fileToProcess}: {testarray.shape}")

            # 判斷維度並解包
            if testarray.size == 0 or len(testarray.shape) < 3:
                print(f"ERROR: No image data or unexpected array dimensions {testarray.shape} in {fileToProcess}")
                return
            elif len(testarray.shape) == 4:
                frames, height, width, channels = testarray.shape
            elif len(testarray.shape) == 3:
                frames, height, width = testarray.shape
                channels = 1  # 假設單通道

        except Exception as e:
            print(f"Failed to read DICOM file {fileToProcess}: {e}")
            return

        # Process image array
        try:
            frame0 = testarray[0]
            
            # 檢查 frame0 是否為單維度並進行計算
            if frame0.ndim == 2:
                mean = np.mean(frame0, axis=1)
            elif frame0.ndim == 3:
                mean = np.mean(frame0, axis=(1, 2))
            else:
                print(f"Unexpected frame dimensions for {fileToProcess}: {frame0.shape}")
                return
            
            # 檢查 np.where 的結果是否非空
            yCrop_positions = np.where(mean < 1)[0]
            if yCrop_positions.size > 0:
                yCrop = yCrop_positions[0]
                testarray = testarray[:, yCrop:, :, :] if channels > 1 else testarray[:, yCrop:, :]
            else:
                print(f"No pixels found with mean < 1 in {fileToProcess}. Skipping yCrop adjustment.")
            
            bias = int(np.abs(testarray.shape[2] - testarray.shape[1]) / 2)
            if bias > 0:
                if testarray.shape[1] < testarray.shape[2]:
                    testarray = testarray[:, :, bias:-bias, :] if channels > 1 else testarray[:, :, bias:-bias]
                else:
                    testarray = testarray[:, bias:-bias, :, :] if channels > 1 else testarray[:, bias:-bias, :]

        except Exception as e:
            print(f"Error processing pixel array of {fileToProcess}: {e}")
            return

        # Set frame rate
        fps = 30
        try:
            fps = dataset[(0x18, 0x40)].value
        except KeyError:
            print("Couldn't find frame rate, default to 30")

        # VideoWriter setup
        try:
            fourcc = cv2.VideoWriter_fourcc(*'MJPG')
            out = cv2.VideoWriter(video_filename, fourcc, fps, cropSize)
        except Exception as e:
            print(f"Error setting up VideoWriter for {fileToProcess}: {e}")
            return

        # Write video frames
        for i in range(frames):
            outputA = testarray[i, :, :, 0] if channels > 1 else testarray[i, :, :]
            smallOutput = outputA[int(height / 10):(height - int(height / 10)), int(height / 10):(height - int(height / 10))]
            output = cv2.resize(smallOutput, cropSize, interpolation=cv2.INTER_CUBIC)
            finaloutput = mask(output)
            finaloutput = cv2.merge([finaloutput, finaloutput, finaloutput])
            out.write(finaloutput)
        
        out.release()
        print(f"Video saved: {video_filename}")

    except Exception as e:
        print(f"An unexpected error occurred while processing {fileToProcess}: {e}")
    return 0

In [ ]:
count = 0

if os.path.exists(AllA4cNames):
    subfolders = os.listdir(AllA4cNames)
    for folder in subfolders:
        folder_path = os.path.join(AllA4cNames, folder)
        
        if os.path.isdir(folder_path):  # 如果是資料夾，繼續進行目錄遍歷
            print(f"Entering folder: {folder}")
            for content in os.listdir(folder_path):
                content_path = os.path.join(folder_path, content)
                if os.path.isdir(content_path):  # 處理內容為資料夾的情況
                    for subcontent in os.listdir(content_path):
                        VideoPath = os.path.join(content_path, subcontent)
                        print(count, folder, content, subcontent)
                        if not os.path.exists(os.path.join(destinationFolder, subcontent + ".avi")):
                            makeVideo(VideoPath, destinationFolder, AllA4cNames)
                        else:
                            print("Already did this file", VideoPath)
                        count += 1
                else:  # 若 content 是檔案
                    print(f"Processing file: {content_path}")
                    if not os.path.exists(os.path.join(destinationFolder, content + ".avi")):
                        makeVideo(content_path, destinationFolder, AllA4cNames)
                    else:
                        print("Already did this file", content_path)
                    count += 1
        else:  # 若 folder_path 是檔案
            print(f"Processing file: {folder_path}")
            if not os.path.exists(os.path.join(destinationFolder, folder + ".avi")):
                makeVideo(folder_path, destinationFolder, AllA4cNames)
            else:
                print("Already did this file", folder_path)
            count += 1

    print("Total count:", count)
else:
    print(f"Directory {AllA4cNames} does not exist.")